**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet 

**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Changing cfg file 
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg #Batch_size
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg #subdivisions
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg #max_batches = class * 2000
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg # no_ classes
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg #
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg #
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg #no_filters = (class + 5)*3
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg 
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4) Extract Images**

The images need to be inside a zip archive called "res.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
####### save images as res.zip and save in mydrive/yolov3/res.zip ######

In [ ]:
!unzip /mydrive/yolov3/res.zip -d data/obj

In [ ]:
full_path_to_images = '/content/darknet/data/obj/res'
c = 0
with open(full_path_to_images + '/' + 'classes.names', 'w') as names, \
     open(full_path_to_images + '/' + 'classes.txt', 'r') as txt:
    for line in txt:
        names.write(line)
        c += 1
with open(full_path_to_images + '/' + 'labelled_data.data', 'w') as data:
    data.write('classes = ' + str(c) + '\n')
    data.write('train = ' + full_path_to_images + '/' + 'train.txt' + '\n')
    data.write('valid = ' + full_path_to_images + '/' + 'test.txt' + '\n')
    data.write('names = ' + full_path_to_images + '/' + 'classes.names' + '\n')
    data.write('backup = backup')

In [ ]:
import os
full_path_to_images = '/content/darknet/data/obj/res'
os.chdir(full_path_to_images)
p = []

for current_dir, dirs, files in os.walk('.'):
    for f in files:
        if f.endswith('.jpg'):
            path_to_save_into_txt_files = full_path_to_images + '/' + f
            p.append(path_to_save_into_txt_files + '\n')

p_test = p[:int(len(p) * 0.15)]

p = p[int(len(p) * 0.15):]

with open('train.txt', 'w') as train_txt:
    for e in p:
        train_txt.write(e)
with open('test.txt', 'w') as test_txt:
    for e in p_test:
        test_txt.write(e)

**6) Start the training**

In [ ]:
# Start the training
!/content/darknet/darknet detector train /content/darknet/data/obj/res/labelled_data.data /content/darknet/cfg/yolov3_training.cfg /content/darknet/darknet53.conv.74 -dont_show